### Deduplication

In [1]:
# Compact, well-labelled deduplication procedure.
# - Drops duplicates based on (name, address, city)
# - If duplicates exist, prefers the row that has a phone value (keeps the row with phone)
# - Falls back to keeping the first row when none in the group have a phone
# - Adds lightweight normalization for grouping to catch trivial formatting differences
# - Prints a short summary and shows examples of dropped rows
#
# Usage: paste this into your session where `companies_df` is defined; the routine checks
# whether `companies_df` exists and will create a tiny sample if it does not (so you can see it run here).

import pandas as pd
import re
from copy import deepcopy

def _norm_str(s: str) -> str:
    """Normalize string for grouping: lower, collapse whitespace, strip punctuation edges."""
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    # collapse multiple spaces/newlines
    s = re.sub(r"\s+", " ", s)
    # remove leading/trailing punctuation like commas/dots/slashes
    s = re.sub(r"^[\W_]+|[\W_]+$", "", s)
    return s

def dedupe_preferring_phone(df: pd.DataFrame,
                             subset_fields=('name','address','city'),
                             phone_field='phone') -> pd.DataFrame:
    """
    Deduplicate df by subset_fields, preferring rows that have a non-empty phone field.
    Returns a deduplicated DataFrame (a copy).
    """
    df = df.copy().reset_index(drop=True)
    # Normalize keys for grouping
    for f in subset_fields:
        df[f"{f}_k"] = df[f].astype(str).apply(_norm_str)
    # Mark phone presence: non-empty after stripping
    df['_phone_present'] = df[phone_field].fillna("").astype(str).str.strip().ne("")
    # Sorting: prefer rows with phone present; we also keep original order by index as tiebreaker
    # Create sort key: rows with phone_present=True should come first (False -> 1, True -> 0)
    df['_sort_key'] = (~df['_phone_present']).astype(int)
    # Sort by grouping keys then sort_key then original index
    sort_cols = [f"{f}_k" for f in subset_fields] + ['_sort_key',]
    df['_orig_index'] = df.index
    df = df.sort_values(by=sort_cols + ['_orig_index'], ascending=[True]*(len(subset_fields)) + [True, True])
    # Drop duplicates keeping the first (which will be a row with phone if any in the group)
    dedup_subset = [f"{f}_k" for f in subset_fields]
    kept = df.drop_duplicates(subset=dedup_subset, keep='first').sort_values(by='_orig_index').reset_index(drop=True)
    # For reporting: determine dropped rows
    kept_idxs = set(kept['_orig_index'].tolist())
    dropped = df[~df['_orig_index'].isin(kept_idxs)].sort_values(by='_orig_index')
    # Clean helper cols from kept
    cols_to_drop = [c for c in kept.columns if c.endswith('_k') or c in ['_phone_present','_sort_key','_orig_index']]
    kept_clean = kept.drop(columns=cols_to_drop)
    print("=== Deduplication summary ===")
    print(f"Initial rows: {len(df)}")
    print(f"Groups (unique name+address+city): {len(df.drop_duplicates(subset=dedup_subset))}")
    print(f"Kept rows: {len(kept_clean)}")
    print(f"Dropped rows: {len(dropped)}\n")
    # Show examples of dropped rows (up to 8) with reason (no phone present)
    if not dropped.empty:
        # annotate reason
        dropped_report = dropped.copy()
        dropped_report['dropped_reason'] = dropped_report.apply(
            lambda r: "no phone" if not r['_phone_present'] else "lower priority (phone exists in group)", axis=1)
        display_cols = [c for c in dropped_report.columns if not c.endswith('_k') and c not in ['_phone_present','_sort_key','_orig_index']]
        print("Examples of dropped rows (up to 8):")
        print(dropped_report[display_cols + ['dropped_reason']].head(8).to_string(index=False))
    else:
        print("No rows were dropped.")
    print("=============================\n")
    return kept_clean.reset_index(drop=True), dropped.loc[:, [c for c in dropped.columns if not c.endswith('_k') and c not in ['_phone_present','_sort_key','_orig_index']]]

# Load the dataset
companies_df = pd.read_csv('output/zimbabwe_companies_20251204_000743.csv')
try:
    companies_df  # attempt to reference
except NameError:
    print("companies_df not found in this environment — creating a small sample mimicking your data for demonstration.\n")

# Run dedupe function
deduped_df, dropped_df = dedupe_preferring_phone(companies_df, subset_fields=('name','address','city'), phone_field='phone')

# Show top rows of deduped result
print("=== Deduped DataFrame (first 10 rows) ===")
print(deduped_df.head(10).to_string(index=False))

# Optionally assign back to companies_df for further processing in your session
companies_df = deduped_df
# Save dropped rows for inspection
dropped_df.to_csv("dropped_rows_examples.csv", index=False)
companies_df.to_csv("cleaned_companies.csv", index=False)
print("\nDropped rows (examples) saved to: dropped_rows_examples.csv")

=== Deduplication summary ===
Initial rows: 42347
Groups (unique name+address+city): 31829
Kept rows: 31829
Dropped rows: 10518

Examples of dropped rows (up to 8):
                                         name                                                                        address     city phone email website description                                                                              source_url  country            crawl_timestamp dropped_reason dropped_reason
    Caroline Garments P/L t/a Caroline Safety                      Address: 26 Bon Accord Rd, Westondale, Bulawayo, Zimbabwe Bulawayo   NaN   NaN     NaN         NaN                              https://www.zimbabweyp.com/company/88429/Caroline_Garments Zimbabwe 2025-12-03T17:58:48.046562       no phone       no phone
               Pest Portal Zimbabwe (Pvt) Ltd Address: Office 11, 3D Building Strathaven Building, Avondale Harare, Zimbabwe   Harare   NaN   NaN     NaN         NaN                          http

In [2]:
companies_df

,name,address,city,phone,email,website,description,source_url,country,crawl_timestamp
0,basic,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zimbabweyp.com/signup-business/basic,Zimbabwe,2025-12-03T17:58:44.209577
1,Caroline Garments P/L t/a Caroline Safety,"Address: 26 Bon Accord Rd, Westondale, Bulaway...",Bulawayo,+263 29 22776452003,NaN,NaN,NaN,https://www.zimbabweyp.com/company/88429/Carol...,Zimbabwe,2025-12-03T17:58:48.045811
2,Pest Portal Zimbabwe (Pvt) Ltd,"Address: Office 11, 3D Building Strathaven Bui...",Harare,07725933442007,NaN,NaN,NaN,https://www.zimbabweyp.com/company/104158/Pest...,Zimbabwe,2025-12-03T17:58:48.050744
3,Busy Bee Transcribing,"Address: 13 Ascot Road, Avondale West, Harare ...",Harare,0242-3089692012,NaN,NaN,NaN,https://www.zimbabweyp.com/company/106087/Busy...,Zimbabwe,2025-12-03T17:58:48.055383
4,Athol Evans Hospita Home,"Address: P O Box Cr 70 Cranb Cranborne, Harare...",Harare,570 8875,NaN,NaN,NaN,https://www.zimbabweyp.com/company/14492/Athol...,Zimbabwe,2025-12-03T17:58:48.057132
...,...,...,...,...,...,...,...,...,...,...
31824,The Bishop's Mistress,NaN,NaN,2011-2025,NaN,NaN,NaN,https://thedirectory.co.zw/company.cfm?company...,Zimbabwe,2025-12-04T00:07:08.834111
31825,Jaipur Restaurant,NaN,NaN,NaN,NaN,https://thedirectory.co.zw/company.cfm?company...,Indian Restaurant,https://thedirectory.co.zw/company.cfm?company...,Zimbabwe,2025-12-04T00:07:22.114354
31826,Windsor Technology,NaN,NaN,NaN,NaN,https://thedirectory.co.zw/company.cfm?company...,Servicing and Installation of Air Conditioners,https://thedirectory.co.zw/company.cfm?company...,Zimbabwe,2025-12-04T00:07:22.190884
31827,Fire Consult,NaN,NaN,NaN,NaN,https://thedirectory.co.zw/company.cfm?company...,"A Total Fire Protection Solution, Automatic Fi...",https://thedirectory.co.zw/company.cfm?company...,Zimbabwe,2025-12-04T00:07:28.297518
